In [ ]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd


ticket = 'KEPL3'
msft = yf.Ticker(ticket + '.SA')

data = msft.info
current_price = data.get('currentPrice', None)
target_high = data.get('targetHighPrice', None)
target_low = data.get('targetLowPrice', None)
target_mean = data.get('targetMeanPrice', None)


if current_price and target_high and target_low and target_mean:

    differences = {
        'Target Low': (target_low - current_price) / current_price * 100,
        'Target Mean': (target_mean - current_price) / current_price * 100,
        'Target High': (target_high - current_price) / current_price * 100,
    }

    price_comparison = pd.DataFrame({
        'Type': ['Current Price', 'Target Low', 'Target Mean', 'Target High'],
        'Price': [current_price, target_low, target_mean, target_high],
        'Difference (%)': [0] + [round(differences[key], 2) for key in differences.keys()]
    })


    fig = go.Figure()

    for i, row in price_comparison.iterrows():
        color = 'blue' if row['Type'] == 'Current Price' else ('green' if row['Difference (%)'] > 0 else 'red')
        fig.add_trace(go.Bar(
            x=[row['Type']],
            y=[row['Price']],
            marker_color=color,
            text=f"{row['Difference (%)']}%",
            textposition='outside',
            name=row['Type']
        ))

    fig.update_layout(
        title=f'Price Comparison and Percentage Difference for {ticket}',
        xaxis_title='Type',
        yaxis_title='Price (BRL)',
        template='plotly_white',
        font=dict(size=14),
        showlegend=False,
        barmode='group'
    )

    fig.show()
else:
    print("Dados insuficientes para gerar o gráfico. Verifique se os preços-alvo e o preço atual estão disponíveis.")


In [ ]:
#result : year: 2025, stock: KEPL3.SA, Low: 26%, Mean: 47%, High: 68.07%